# Compiling and Submitting Circuits onto Hilbert via Cirq


<a target="_blank" href="https://colab.research.google.com/github/SupertechLabs/client-superstaq/blob/main/docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" align="left"/>
</a>
<a target="_blank" href="https://mybinder.org/v2/gh/SupertechLabs/client-superstaq/HEAD?labpath=docs/source/optimizations/cq/cq_compile_css.ipynb">
  <img src="https://mybinder.org/badge_logo.svg" alt="Launch Binder" align="left" style="margin-left: 10px;"/>
</a>

This tutorial will showcase how to compile and submit a circuit onto Infleqtion's hardware, Hilbert, using the cirq-superstaq client. First, you must import the necessary packages to get started. 

## Import Requirements

In [2]:
import os
import json

try:
    import cirq
    import cirq_superstaq as css

except ImportError:
    print("installing cirq-superstaq...")
    !pip install --quiet cirq-superstaq
    import cirq
    import cirq_superstaq as css

    print("installed cirq-superstaq")

Next, we must instantiate our provider in cirq-superstaq, using a SuperstaQ token. You can get your SuperstaQ token from https://superstaq.super.tech. 

In [3]:
token = "
service = css.Service(token)

## Create a Circuit

In [4]:
qubits = cirq.LineQubit.range(2)
circuit1 = cirq.Circuit(cirq.H(qubits[0]), cirq.CNOT(qubits[0], qubits[1]), cirq.measure(qubits[0]))
print(circuit1)

0: ───H───@───M───
          │
1: ───────X───────


## Single Circuit Compilation

In [5]:
compiler_output = service.cq_compile(circuit1)

In [6]:
print(compiler_output.circuit)

(0, 0): ───RGate(0.5π, -0.5π)───@───RGate(-0.25π, -0.5π)───────RGate(0.25π, -0.5π)───────M('q(0)')───
           │                    │   │                          │
(0, 1): ───#2───────────────────@───#2─────────────────────Z───#2────────────────────Z───────────────


To get your ColdQuanta credentials onto your local machine, run this:

## Single Circuit Submission

The code below will submitt he circuit to the noiseless Hilbert simulator. If you would like to run on Hilbert, change the "target" argument in job_css from "cq_hilbert_simulator" to "cq_hilbert_qpu". It is recommended to first submit to the simulator for testing to ensure your code runs before submitting to Hilbert. 

In [8]:
job_css = service.create_job(circuit=circuit1, repetitions=100, target="cq_hilbert_simulator")
result_css = job_css.counts()
print(f"Counts from cirq-superstaq submission: {result_css}")

Counts from cirq-superstaq submission: {'1': 54, '0': 46}


## Multiple circuit compilation

In [7]:
compiler_output = service.cq_compile([circuit, circuit1])

In [8]:
print(compiler_output.circuits[0])
print()
print(compiler_output.circuits[1])

(0, 0): ───RGate(0.5π, -0.5π)───M───

(0, 0): ───RGate(0.5π, -0.5π)───@───M───RGate(-0.25π, -0.5π)───────────RGate(0.25π, -0.5π)───────────
           │                    │       │                              │
(0, 1): ───#2───────────────────@───────#2─────────────────────Rz(π)───#2────────────────────Rz(π)───
